In [0]:
# Input files: word2vec model, paragraph embedding file, paragraph files, input question file
# ouput: json file with selected paragraph

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import re
import json

In [0]:
basePath = '/content/'

inpFile = basePath+'traindata6.json'
paraFile = basePath+'temp.json'
paraEmbFile = basePath+'paraEmbs.csv'

outFile = basePath+'out.json'

In [0]:
'''import zipfile
import os
for file_name in os.listdir(drive_dir):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"import zipfile\nimport os\nfor file_name in os.listdir(drive_dir):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(drive_dir+'/'+file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [0]:
def sifEmbeddings(sentences, model, alpha=1e-3):

    vocab = model.wv.vocab
    embs = model.wv       
    size = model.vector_size  
    
    total = 0
    for word in vocab:
        total += vocab[word].count 

    output = []
    
    for sent in sentences:
        count = 0
        v = np.zeros(size, dtype=np.float32) 
        for word in sent:
          if word in vocab:
            #print(word)
            weight = alpha / (alpha + (vocab[word].count/total))
            v += weight * embs[word]
            count += 1 
                
        if count > 0:
            v *= 1/count

        output.append(v)
    return np.vstack(output)

In [0]:
def splitSent(sent):
    regX = '[\s()\.\?]'
    splitSent = re.split(regX,sent)
    splitSent = list(filter(None,splitSent))
    return splitSent

In [0]:
def getFileSent(inpFile):
    fileSent = ''
    with open(inpFile,'r') as f:
        x = json.loads(f.read())
        for option in x['options'][0]:
            fileSent += option
            fileSent += ' '
        fileSent += x['questions'][0]
    return fileSent

In [0]:
def getMatchingPara(quesEmb):
    paraEmbs = np.loadtxt('paraEmbs.csv',delimiter=',')

    scores = []
    for paraEmb in paraEmbs:
        score=float( cosine_similarity([paraEmb],[emb]) )
        scores.append(score)
    print(scores)
    mxScoreInd = scores.index(max(scores))
    mxScoreInd += 1

    with open(paraFile,'r') as f:
      x = json.loads(f.read())
      return x['para'+str(mxScoreInd)]

In [0]:
def getQuesEmb(model,inpFile):
    fileSent = getFileSent(inpFile)
    sent = splitSent(fileSent)
    emb = sifEmbeddings([sent], model, alpha=1e-3)[0]
    return emb

In [0]:
def saveFile(inpFile,outFile,para):
    with open(inpFile,'r') as f:
      x = json.loads(f.read())
      x['article'] = para

      with open(outFile, 'w') as jsonOut:
          json.dump(x, jsonOut)

In [0]:
model = Word2Vec.load("word2vec.model")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
emb = getQuesEmb(model,inpFile)
print(emb)

[ 1.0139591e-03 -1.3389041e-04 -1.2687899e-03 -1.0514830e-03
 -5.7841063e-04  1.4504138e-03 -1.7172951e-03  1.9267428e-04
 -2.8005318e-04  6.7244287e-06]


In [0]:
para = getMatchingPara(emb)
print(para)

[0.3510968102007468, 0.43503507329196806, 0.26197185403590006, 0.5152197942059374, 0.01810624687591992, 0.5073791943771575]
All radioactive decay is dangerous to living things, but alpha decay is the least dangerous.


In [0]:
saveFile(inpFile,outFile,para)